## Deliverable 2. Create a Customer Travel Destinations Map.

In [36]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [37]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thompson,CA,55.7435,-97.8558,-20.04,69,75,4.61,broken clouds
1,1,Airai,TL,-8.9266,125.4092,72.25,72,91,1.63,overcast clouds
2,2,Provideniya,RU,64.3833,-173.3000,-15.38,90,4,8.68,clear sky
3,3,Changji,CN,44.0167,87.3167,14.79,92,75,2.24,mist
4,4,Atuona,PF,-9.8000,-139.0333,78.08,75,8,12.77,clear sky


In [38]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [39]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Atuona,PF,-9.8000,-139.0333,78.08,75,8,12.77,clear sky
5,5,Rikitea,PF,-23.1203,-134.9692,79.18,74,5,16.93,clear sky
6,6,Kapaa,US,22.0752,-159.3190,78.78,71,75,4.61,broken clouds
8,8,Avarua,CK,-21.2078,-159.7750,82.45,74,20,6.91,few clouds
15,15,Hobart,AU,-42.8794,147.3294,75.02,46,75,14.97,broken clouds
22,22,Geraldton,AU,-28.7667,114.6000,84.61,51,30,17.27,scattered clouds
29,29,Bathsheba,BB,13.2167,-59.5167,77.61,61,20,23.02,few clouds
34,34,Dubbo,AU,-32.2500,148.6167,82.53,32,40,12.66,scattered clouds
36,36,Cabedelo,BR,-6.9811,-34.8339,79.47,81,62,10.07,broken clouds
38,38,Vaini,TO,-21.2000,-175.2000,84.36,89,20,8.05,few clouds


In [40]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                404
City                   404
Country                404
Lat                    404
Lng                    404
Max Temp               404
Humidity               404
Cloudiness             404
Wind Speed             404
Current Description    404
dtype: int64

In [44]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_prefcities_df = preferred_cities_df.dropna()

In [45]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_prefcities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,78.08,clear sky,-9.8000,-139.0333,
5,Rikitea,PF,79.18,clear sky,-23.1203,-134.9692,
6,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,
8,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,
15,Hobart,AU,75.02,broken clouds,-42.8794,147.3294,
22,Geraldton,AU,84.61,scattered clouds,-28.7667,114.6000,
29,Bathsheba,BB,77.61,few clouds,13.2167,-59.5167,
34,Dubbo,AU,82.53,scattered clouds,-32.2500,148.6167,
36,Cabedelo,BR,79.47,broken clouds,-6.9811,-34.8339,
38,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,


In [46]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [59]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,78.08,clear sky,-9.8000,-139.0333,Villa Enata
5,Rikitea,PF,79.18,clear sky,-23.1203,-134.9692,People ThankYou
6,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,Paradise Inn
15,Hobart,AU,75.02,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
...,...,...,...,...,...,...,...
1974,Caruray,PH,82.27,broken clouds,10.3094,119.0128,
1975,Kendari,ID,81.48,overcast clouds,-3.9450,122.4989,Hotel Wixel
1976,Ambovombe,MG,78.73,clear sky,-25.1667,46.0833,RELAHARA
1981,Samarinda,ID,76.39,overcast clouds,-0.5000,117.1500,ASTON Samarinda Hotel & Convention Center


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
